In [ ]:
# Loading necessary libraries.
import requests
import csv

In [ ]:
# Defining the API key for Google geocoding servide.
api_key = 'enter here API key'

# Entering file path.
file_path = 'enter here file path'

# Creating an empty list.
updated_rows = []

In [ ]:
# Looping through the locations and matching them with coordinates, if possible.
coordinates_count = 0
no_coordinates_count = 0

with open(file_path, 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    header = next(reader)
    header.extend(["lat", "lon"])
    updated_rows.append(header)

    for row in reader:
        address = row[0]
        url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
        response = requests.get(url)
        data = response.json()

        if data["status"] == "OK":
            location = data["results"][0]["geometry"]["location"]
            latitude = location["lat"]
            longitude = location["lng"]
            row.extend([latitude, longitude])
            coordinates_count += 1
        elif data["status"] == "ZERO_RESULTS":
            print(f"No coordinates found for {address}.")
            row.extend(["", ""])
            no_coordinates_count += 1
        else:
            print(f"Error for {address}: {data['status']}")
            row.extend(["", ""])
            no_coordinates_count += 1

        updated_rows.append(row)

output_file_path = 'teatajapoliit_40s_coordinates.txt'
with open(output_file_path, 'w', newline='', encoding='utf-8') as output_file:
    writer = csv.writer(output_file, delimiter=',')
    writer.writerows(updated_rows)

total_places = coordinates_count + no_coordinates_count
coordinates_percentage = (coordinates_count / total_places) * 100
no_coordinates_percentage = (no_coordinates_count / total_places) * 100

print(f"Updated data written to {output_file_path}")
print(f"Coordinates found for {coordinates_count} place names ({coordinates_percentage:.2f}%).")
print(f"No coordinates found for {no_coordinates_count} place names ({no_coordinates_percentage:.2f}%).")